In [ ]:
import pandas as pd
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Ce dont on a besoin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [ ]:
chemin_bd = r"./bd_ignore/"

In [ ]:
df_merge_recom = pd.read_csv(chemin_bd+"resultat2/merge_recom.csv")

In [ ]:
df_merge_recom.info()

In [ ]:
df_merge_recom

In [ ]:
df_merge_recom21 = df_merge_recom.drop(columns=['adult', 'original_language', 'spoken_languages', 
                                                'tagline', 'title', 'originalTitle', 'genres_x', 'revenue', 
                                                'release_date', 'primaryProfession'])

In [ ]:
df_merge_recom21.to_csv(chemin_bd+"resultat2/df_merge_recom21.csv", index=False)

In [ ]:
df_merge_recom22 = pd.read_csv(chemin_bd+"resultat2/df_merge_recom21.csv")

In [ ]:
df_merge_recom22.isna().sum()

In [ ]:
df_merge_recom22['homepage'] = df_merge_recom22['homepage'].fillna('None')
df_merge_recom22['poster_path'] = df_merge_recom22['poster_path'].fillna('None')
df_merge_recom22['overview'] = df_merge_recom22['overview'].fillna('None')

In [ ]:
df_merge_recom22.info()

In [ ]:
caracteristiques = ['titleType', 'startYear', 'genres_y', 'primaryName']

In [ ]:
merge_filmR = df_merge_recom22[df_merge_recom22['titleType'] == 'movie']
merge_serieR = df_merge_recom22[df_merge_recom22['titleType'] == 'tvSeries']

In [ ]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  return X_encoded, SN

In [ ]:
X = merge_filmR[caracteristiques]
# ne prend que les colonnes qui sont dans caracteristiques

X_encoded, SN = encodage_X(X, type='normalisation')

In [ ]:
# on met toutes les données dans une colonne et de maniere unique avec .set

In [ ]:
df_merge_recom22 = df_merge_recom22.groupby(
    'imdb_id', sort=False, as_index=False
).agg({
    'primaryName': set,  # Affiche uniquement des valeurs uniques
    'genres_y': set,
    'startYear': set,
    'titleType': set,
    'poster_path': set,
    'overview': set,
    'original_title': set,  # Applique automatiquement l'unicité
    'homepage':set
    
})



In [ ]:
df_merge_recom22.info()

In [ ]:
df_merge_recom22['primaryName']=df_merge_recom22['primaryName'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['genres_y']=df_merge_recom22['genres_y'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['startYear']=df_merge_recom22['startYear'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['titleType']=df_merge_recom22['titleType'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['poster_path']=df_merge_recom22['poster_path'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['overview']=df_merge_recom22['overview'].apply(lambda x : str(x).replace('{', '').replace('}',''))
df_merge_recom22['original_title']=df_merge_recom22['original_title'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['homepage']=df_merge_recom22['homepage'].apply(lambda x : str(x).replace('{','').replace('}',''))


In [ ]:
df_merge_recom22['primaryName']=df_merge_recom22['primaryName'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['genres_y']=df_merge_recom22['genres_y'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['startYear']=df_merge_recom22['startYear'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['titleType']=df_merge_recom22['titleType'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['poster_path']=df_merge_recom22['poster_path'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['overview']=df_merge_recom22['overview'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['original_title']=df_merge_recom22['original_title'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['homepage']=df_merge_recom22['homepage'].apply(lambda x : str(x).replace("'",'').replace("'",''))

In [ ]:
df_merge_recom22

In [ ]:
# standardidation des données

In [ ]:
df_merge_recom222 = df_merge_recom22.copy()

In [ ]:
df_merge_recom222.replace('nan', np.nan)

In [ ]:
df_merge_recom222.apply(lambda col: col.str.contains('NaN', na=False)).any().any()

In [ ]:
# Trouver les lignes et colonnes contenant 'NaN'
nan_positions = df_merge_recom222.apply(lambda col: col.str.contains('NaN', na=False))

# Afficher les positions
rows_with_nan = nan_positions.any(axis=1)  # Lignes avec 'NaN'
cols_with_nan = nan_positions.any(axis=0)  # Colonnes avec 'NaN'

print("Lignes contenant 'NaN':", df_merge_recom222[rows_with_nan])
print("Colonnes contenant 'NaN':", cols_with_nan[cols_with_nan].index.tolist())

In [ ]:
#df_merge_recom222.drop(row:)

In [ ]:
X= df_merge_recom22.drop(columns=['imdb_id','primaryName'])
y=	df_merge_recom22['imdb_id']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
df_merge_recom22.info()

In [ ]:
#essai5

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_merge_recom22['imdb_normalise'] = scaler.fit_transform(df_merge_recom22[['imdb_id']])

In [ ]:
# similarite

In [ ]:
RecomX= df_merge_recom22.drop(columns=['imdb_id','primaryName'])

In [ ]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  return X_encoded, SN

In [ ]:
X_encoded, SN = encodage_X()

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Création et ajustement du modèle NearestNeighbors
nn = NearestNeighbors(n_neighbors=3, metric='euclidean')
nn.fit(X_encoded)

imdb2 = ['6061', '19474']

for name in champion_legendary :
  # Recherche des films similaires à un imdb2
  df_merge_recom22.index =   df_merge_recom22[df_merge_recom22['name'] == name].index[0]
[df_merge_recom22['name'] == name].index[0] 
recom_feature = X_encoded.iloc[df_merge_recom22.index].values.reshape(1, -1)
  # Recherche des plus proches voisins
distances, indices = nn.kneighbors(recom_feature)
print(f"\nFilms les plus similaires à un {name}:")
for distance, index in zip(distances[0][1:], indices[0][1:]):
    print(f"{df_merge_recom22['Name'].iloc[index]} (Distance: {distance:.2f})")
    recom_film = df_merge_recom22.iloc[index]
    print(recom_film)